<html>
  <head>
    <title>Hello PyLadies!</title>
  </head>
  <body>
    <h2 class='opening'>Welcome to my <a href="http://melbourne.pyladies.com/">PyLadies Melbourne</a> talk</h2>
      <p>A few comments:</p>
      <li id='new'>I'm pretty new to this</li>
      <li id='fun'>But it's been fun to learn</li>
      <p></p>
    <p class = '1'>Hopefully you'll learn something. </p>
    <div>There is a lot to learn. Certainly there is much I do not know.
        <a href="https://www.brainyquote.com/topics/learning-quotes">But that's okay.
        </a>
    </div>
    <p class = '2'>Hopefully I'll learn something new.</p>
    <p class = '3'>And we'll all have some fun. Because learning is often fun!</p>
      <div>If you missed, or want to review, the
          <a href='https://docs.google.com/presentation/d/1u7HDawQFpMzj-2epPIdKNysLOl_lc0BoNam4dApnMlM/edit?usp=sharing'>intro presentation</a>
          you can find it here.
      </div>
    <p class = 'closing'>Let's get started!<p>
  </body>
</html>


In [1]:
#imports
from scrapy.selector import Selector

import requests
from bs4 import BeautifulSoup

import pandas as pd


## A little taste of Scrapy & XPath
XPath locates items within html by following the tag structure of the code

In [2]:
html_str = '''
<html>
  <head>
    <title>Hello PyLadies!</title>
  </head>
  <body>
    <h2 class='opening'>Welcome to my <a href="http://melbourne.pyladies.com/">PyLadies Melbourne</a> talk</h2>
      <p>A few comments:</p>
      <li id='new'>I'm pretty new to this</li>
      <li id='fun'>But it's been fun to learn</li>
      <p></p>
    <p class = '1'>Hopefully you'll learn something. </p>
    <div>There is a lot to learn. Certainly there is much I do not know.
        <a href="https://www.brainyquote.com/topics/learning-quotes">But that's okay.
        </a>
    </div>
    <p class = '2'>Hopefully I'll learn something new.</p>
    <p class = '3'>And we'll all have some fun. Because learning is often fun!</p>
      <div>If you missed, or want to review, the
          <a href='https://docs.google.com/presentation/d/1u7HDawQFpMzj-2epPIdKNysLOl_lc0BoNam4dApnMlM/edit?usp=sharing'>intro presentation</a>
          you can find it here.
      </div>
    <p class = 'closing'>Let's get started!<p>
  </body>
</html>
'''

In [3]:
xpath_selector = Selector(text=html_str)

In [4]:
print(xpath_selector.xpath('/html/body/h2/text()').extract())

['Welcome to my ', ' talk']


In [5]:
#keep track of your single/double quotes when working with html :)
path = '//*[@class = "opening"]/text()'
xpath_selector.xpath(path).extract()

['Welcome to my ', ' talk']

In [6]:
path1 = '/html/body/li/text()'
path2 = '//*[@id = "new"]/text()'
print(xpath_selector.xpath(path1).extract())
print(xpath_selector.xpath(path2).extract())

["I'm pretty new to this", "But it's been fun to learn"]
["I'm pretty new to this"]


In [ ]:
path3 = '/html/body/p[@class = 3]/text()'
xpath_selector.xpath(path3).extract()

In [7]:
paragraphs = [xpath_selector.xpath('/html/body/p[@class = ' + str(i+1) +']/text()').extract()
                  for i in range(3)]
print(paragraphs)

[["Hopefully you'll learn something. "], ["Hopefully I'll learn something new."], ["And we'll all have some fun. Because learning is often fun!"]]


## Requests
Let's get some html!

In [8]:
#using requests to capture html from a website
response = requests.get('https://www.python.org/')
response.status_code

200

### HTML status codes
These responses tell about whether your request worked, or what might have gone wrong. You've probably seen 404 frequently. You don't usually see 200 because it means things are all good <br>
<br>
learn more: https://en.wikipedia.org/wiki/List_of_HTTP_status_codes <br>
<br>


In [9]:
#let's look at what's in the response
#how long is it?
print(len(response.text))

#what does it look like?
response.text[0:1000]

49055


'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">\n\n    <meta name="application-name" content="Python.org">\n    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">\n    <meta name="apple-mobile-web-app-title" content="Python.org">\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="apple-mobile-web-app-status-bar-style" content="black">\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="Han

In [10]:
response.apparent_encoding

'ascii'

In [11]:
xpath_selector2 = Selector(response)

In [19]:
#let's find the events
xpath_selector2.xpath('//*[@id="content"]/div/section/div[2]/div[2]/div/ul/li[1]/a/text()').extract()[0]

'PyCon TW 2019'

## BeautifulSoup

In [20]:
soup = BeautifulSoup(html_str)

In [25]:
soup.find('div')
soup.find('div').text.strip()

soup.find_all('div')
soup.find_all('div').text

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [30]:
soup.find_all('a')

links = [link.text
         for link in soup.find_all('a')
         if link.has_attr('href')]        

print(links)

links = [] 
for link in soup.find_all('a'):
    if link.has_attr('href'):
        links.append(link['href'])

print(links)

['PyLadies Melbourne', "But that's okay.\n        ", 'intro presentation']
['http://melbourne.pyladies.com/', 'https://www.brainyquote.com/topics/learning-quotes', 'https://docs.google.com/presentation/d/1u7HDawQFpMzj-2epPIdKNysLOl_lc0BoNam4dApnMlM/edit?usp=sharing']


### Let's check out a real website

In [31]:
quotes_url = requests.get(links[1])
soup_quotes = BeautifulSoup(quotes_url.text)

In [33]:
#who are the authors?
for a in soup_quotes.find_all('a',{'class':"bq-aut qa_159029 oncl_a"}):
    print(a.text)
for a in soup_quotes.find_all('a',{'title':"view author"}):
    print(a.text)

Earl Nightingale
Earl Nightingale
Pele
Tena Desae
Colin Powell
Brad Henry
Anthony J. D'Angelo
Steve Harvey
Amisha Patel
Billy Graham
Jim Rohn
Maya Angelou
Eliphas Levi
Colin Powell
Benjamin Franklin
Jiddu Krishnamurti
Henry Ford
Michelle Obama
George Foreman
Wilma Rudolph
Elisabeth Kubler-Ross
Asa Hutchinson
Mahatma Gandhi
Joel Edgerton
Gza
Bruce Lee
Queen Latifah


In [39]:
#let's get the keywords for the quotes
soup_quotes.find_all('a', {'class':"qkw-btn btn btn-xs oncl_list_kc"})

[<a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="0" href="/topics/life-quotes">Life</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="1" href="/topics/work-quotes">Work</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="2" href="/topics/family-quotes">Family</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="3" href="/topics/future-quotes">Future</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="0" href="/topics/hard-work-quotes">Hard Work</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="1" href="/topics/work-quotes">Work</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="2" href="/topics/love-quotes">Love</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="3" href="/topics/success-quotes">Success</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="0" href="/topics/positive-quotes">Positive</a>,
 <a class="qkw-btn btn btn-xs oncl_list_kc" data-idx="1" href="/topics/work-quotes">Work</a>,
 <a class="qkw-btn btn btn-x

## Working with JavaScript and similar dynamic features
The request object only grabs the html - if something is using javascript it won't see it. One way to get around this is to use automated testing software designed to mimic human browsing habits. I'll be using Selenium and chromedriver, which you can learn about here: <br>
<br>
https://www.seleniumhq.org/ <br>
<br>
https://sites.google.com/a/chromium.org/chromedriver/home <br>
<br>
If you decide to use scrapy for your project, here is an option to play with:<br>
https://github.com/scrapy-plugins/scrapy-splash<br>
<br>

In [40]:
url = '''https://www.opentable.com/s/?covers=2&dateTime=2019-09-16%2019%3A00&metroId=279
        &regionIds=1145&enableSimpleCuisines=true&userlongitude=145.00&userlatitude=-37.83
        &includeTicketedAvailability=true&pageType=0'''

response2 = requests.get(url)
print(response2.status_code)

html = response2.text

200


In [41]:
soup_booking = BeautifulSoup(html,'html.parser')
soup_booking.title.text

'Restaurant Reservation Availability'

In [42]:
soup_booking.find_all('span',{'class':"rest-row-name-text"}) 


[<span class="rest-row-name-text">Mariah Port</span>,
 <span class="rest-row-name-text">Plaza</span>,
 <span class="rest-row-name-text">Wolff</span>,
 <span class="rest-row-name-text">Emerys</span>,
 <span class="rest-row-name-text">Predovic Hill</span>,
 <span class="rest-row-name-text">432 Gaylord</span>,
 <span class="rest-row-name-text">Kareem Schamberger</span>,
 <span class="rest-row-name-text">Quod Parker</span>,
 <span class="rest-row-name-text">Kohler</span>,
 <span class="rest-row-name-text">Raquel Jakubowski</span>,
 <span class="rest-row-name-text">Voluptatem Garden</span>,
 <span class="rest-row-name-text">Agloe Bar &amp; Grill</span>,
 <span class="rest-row-name-text">Excepturi Jakubowski</span>,
 <span class="rest-row-name-text">469 Block</span>,
 <span class="rest-row-name-text">Iste Jaskolski</span>,
 <span class="rest-row-name-text">Voluptatem Purdy</span>,
 <span class="rest-row-name-text">Hollow</span>,
 <span class="rest-row-name-text">Sapiente</span>,
 <span class

In [43]:
soup_booking.find_all('div', {'class':"booking"})

[]

In [45]:
from selenium import webdriver
from time import sleep


In [46]:
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe") 
#tell Selenium where to find the driver

In [47]:
#when you're done with it
driver.close()

In [48]:
def get_html_with_script(url):
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")
    driver.get(url)

    #wait one second
    sleep(1)

    #grab page source - javascript will have been executed
    html = driver.page_source
    driver.close()
    
    return html

In [49]:
html_driver = get_html_with_script(url)
soup_driver = BeautifulSoup(html_driver)
soup_driver.find_all('div', {'class':"booking"})

[<div class="booking"><span class="tadpole"></span>Booked 32 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 14 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 12 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 13 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 24 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 10 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 4 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 4 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 1 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 1 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 7 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 7 times today</div>,
 <div class="booking"><span class="tadpole"></

### A second example: job boards

In [50]:
url2 = 'https://www.seek.com.au/job/39919899'

In [51]:
job_html = requests.get(url2)
job_html.status_code

200

In [52]:
job_soup = BeautifulSoup(job_html.text)
job_soup.find_all('div', {'class':"lmis-cg-salary-bottom"})

[]

In [53]:
job_soup_new = BeautifulSoup(get_html_with_script(url2))
job_soup_new.find_all('div', {'class':"lmis-cg-salary-bottom"})

[<div class="lmis-cg-salary-bottom">
 <span style="width: 75.000%">
             $45K
           </span>
 <span class="lmis-cg-salary-mode">$120K</span>
 <span>$160K</span>
 </div>]

## Final Notes
Think about ethics and laws when planning your project. This is a developing area, so do your research, particularly if you plan to publish or commercialize your work.

### Example of data collection [code](https://github.com/LydiaPeabody/Portfolio/blob/master/Capstone_JD_Gender_Analysis/Indeed_Job_Scraping.ipynb)
This was just my approach - probably not the best approach, but gives you some ideas to work from


### Recognition
Thanks to Srikanta Patra for helping me learn web scraping as part of the General Assembly Data Science Immersive. Some of this code is adapted from his lessons, particularly the use of Open Table as a good example for javascript.<br>
And thanks to Elke Hansen for pointing out the javascript in Seek ads.